# Interacting Particle Markov Chain Monte Carlo
## Introduction & Context

This notebook presents the interacting particle markov chain monte carlo algorithm from http://proceedings.mlr.press/v48/rainforth16.html by Rainforth et al. (Proceedings of the 33rd International Conference on Machine Learning,). This algorithm is an extension of the family of particleMarkov chain Monte Carlo algorithms originally proposed in https://www.stats.ox.ac.uk/~doucet/andrieu_doucet_holenstein_PMCMC.pdf by Andrieu et al. .

### Mathmatical context

We will focus on a Markovian model even if the algorithm is not limited to this type of models.

We have $x_t$ the states of our model and $y_t$ our observation following:
- $ x_t | x_{1:t−1} ∼ f_t(x_t|x_{1:t−1}) $ the transition model 
- $ y_t | x_{1:t} ∼ g_t ( y_t | x_{1:t}) $ the observation model
- $ x_1 ∼ \mu(\cdot)$

Our goal is to compute expectation values with respect to the posterior distribution $ p(x_{1:T}|y_{1:T}) \propto \mu(x_1)\prod\limits_{t=2}^{T}f_t(x_t | x_{1:t-1}) \prod\limits_{t=1}^{T} g_t ( y_t | x_{1:t} ) $

### Difficulties
As explained in Andrieu et al., Sequential Monte Carlo (SMC) and Markov Chain Monte Carlo (MCMC) methods are often unreliable when the proposal distributions that are used to explore the space are poorly chosen and/or if highly correlated variables are updated independently. Particle Markov Chain Monte Carlo (PMCMC) try to solve theses issues using SMC methods to construct efficient proposals for the MCMC sampler. 

Interacting particle Markov Chain Monte Carlo tries to increase the efficiency of PMCMC, and solve its issue with *path degeneracy*.
Whenever the ancestral lineage collapses at the early stages of the state sequence, the common ancestor is, by construction, guaranteed to be equal to the retained particle resulting in sample impoverishment, high correlation between the samples, and poor mixing of the Markov chain.

To make things more concrete let's dive in the algorithm!

## iPMCMC
### Algorithm

To explore the iPMCMC algorithm we must first explore SMC and CSMC, as iPMCMC uses both to generate efficient proposals of $x_{1:T}$ to the MCMC sampler.

#### Sequential Monte Carlo

The main idea of SMC is to generate at each time step a new position for each particle in our system, but instead of generating the next position of a particle with the particle past we take the past of any particle in our system with a discrete distribution based on the normalized weights of each one. 
The resulting particle system approximate $p(x_{1:t}|y_{1:t})$

<img src="./images/algo1.PNG" width=500px></img>

#### Conditional Sequential Monte Carlo

CSMC is basically the same as SMC but fixing one particle trajectory. All other particles can use this fixed past but the particle itself is fixed.

<img src="./images/algo2.PNG" width=500px></img>

#### Interaction Particle Markov Chain Monte Carlo

IPMCMC combines both previous algorithms as MCMC sampler.
We have M workers, P of theses M workers being CSMC samplers and M-P SMC sampler.
At each step $r$ of the sampler all the workers generate their output according to the previously explained protocols.
Then for each CSMC worker we randomize its output between its own and any of the SMC workers, with probabilities:

```Add equations here```

We then choose the fixed trajectory of each CSMC worker with a weighted randomize choice over the new outputs outputs chosen previously.

```Add equations here```

The output of our algorithm is the chains of all CSMC workers chosen fixed particles. That gives us $R \times P$ trajectories.

<img src="./images/algo3.PNG" width=500px></img>

### Results

In [1]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from tqdm import tqdm
from ipmcmc.generate_data import *
from ipmcmc.linear_gaussian_state_model import *
from ipmcmc.non_linear_gaussian_state_model import *
from ipmcmc.smc import *
from ipmcmc.csmc import *
from ipmcmc.ipmcmc import *
from ipmcmc.estimation import *    


We implemented the same models as those in the paper. Each one of them is implemented and generates observations and states in the two next cells

In [2]:
# 4.1. Linear Gaussian State Space Model
np.random.seed(420)
# Parameters
t_max = 50
n_particles = 50

r = R.from_rotvec(np.array([7*np.pi/10, 3*np.pi/10, np.pi/20]))
rotation_matrix = r.as_dcm()
scaling_matrix = 0.99*np.eye(3)
beta = np.random.dirichlet(np.ones(20)*0.2, 3).transpose()
alpha = scaling_matrix@rotation_matrix
t_max = 50
mu = np.array([0, 1, 1])
start_var = 0.1*np.eye(3)
omega = np.eye(3)
sigma = 0.1*np.eye(20)



l_transition_model = [LinearMu(default_mean=mu, default_cov=start_var)]+[LinearTransition(
    default_mean=np.zeros(3), default_cov=omega, default_alpha=alpha) for t in range(1, t_max)]
l_proposals = [LinearMu(default_mean=mu, default_cov=start_var)]+[LinearProposal(
    default_mean=np.zeros(3), default_cov=omega, default_alpha=alpha) for t in range(1, t_max)]
l_observation_model = [LinearObservation(default_mean=np.zeros(
    20), default_cov=sigma, default_beta=beta) for t in range(0, t_max)]

# If we want to change the parameters
assert np.all(np.linalg.eigvals(start_var) > 0)
assert np.all(np.linalg.eigvals(omega) > 0)
assert np.all(np.linalg.eigvals(sigma) > 0)

l_states, l_observations = linear_gaussian_state_space(
    t_max=t_max, mu=mu, start_var=start_var, transition_var=omega, noise_var=sigma,
    transition_coeffs=alpha, observation_coeffs=beta)

In [3]:
# 4.2. Nonlinear State Space Model
np.random.seed(420)
nl_mu = 0
start_std = np.sqrt(5)
omega = np.sqrt(10)
sigma = np.sqrt(10)

nl_transition_model = [NonLinearMu(default_mean=nl_mu, default_std=start_std)]+[NonLinearTransition(
    default_mean=0, default_std=omega) for t in range(1, t_max)]
nl_proposals = [NonLinearMu(default_mean=nl_mu, default_std=start_std)]+[
    NonLinearProposal(default_mean=0, default_std=omega) for t in range(1, t_max)]
nl_observation_model = [NonLinearObservation(
    default_mean=0, default_std=sigma) for t in range(0, t_max)]

nl_states, nl_observations = nonlinear_gaussian_state_space(
    t_max=t_max, mu=nl_mu, start_std=start_std, transition_std=omega, noise_std=sigma)

In [4]:
# ipmcmc run: works with both linear and non-linear models.
# It is pretty long to run, longer for the linear model which has 3-dimensional states.
# For the linear model, each MCMC step take approximately 90 secs, and 80 secs for 
# the non-linear, on our computers.

n_nodes = 32
n_conditional_nodes = 16
n_steps = 10

linear= True

if linear:
    print('init_conditional_traj')
    
    init_conditional_traj = np.zeros((n_conditional_nodes, t_max, len(mu)))
    for i in tqdm(range(n_conditional_nodes)):
        
        particles, weights, _ = smc(l_observations, n_particles,
                              l_transition_model, l_proposals, l_observation_model)
        b_j = np.argmax(weights[-1])
        init_conditional_traj[i] = particles[:,b_j]

    print('running ipmcmc')
    particles, conditional_traj, weights, conditional_indices, zetas = ipmcmc(
        n_steps, n_nodes, n_conditional_nodes, l_observations, n_particles, init_conditional_traj,
        l_proposals, l_transition_model, l_observation_model)

else:
    print('init_conditional_traj')
    
    init_conditional_traj = np.zeros((n_conditional_nodes, t_max, 1))
    for i in tqdm(range(n_conditional_nodes)):
        particles, weights, _ = smc(nl_observations, n_particles,
                              nl_transition_model, nl_proposals, nl_observation_model)
        b_j = np.argmax(weights[-1])
        init_conditional_traj[i] = particles[:,b_j]

    print('running ipmcmc')
    particles, conditional_traj, weights, conditional_indices, zetas = ipmcmc(
        n_steps, n_nodes, n_conditional_nodes, nl_observations, n_particles, init_conditional_traj,
        nl_proposals, nl_transition_model, nl_observation_model)
        

  0%|          | 0/16 [00:00<?, ?it/s]

init_conditional_traj


  0%|          | 0/10 [00:00<?, ?it/s]

running ipmcmc


100%|██████████| 10/10 [08:05<00:00, 48.57s/it]


In [5]:
# Mean estimation for the linear model, using kalman filter and rts smoother as ground truth
# Make sure that the particles used are the one generated during a run of the ipmcmc sampler
# for the linear model

true_means, true_covs = compute_ground_truth(l_observations, mu, start_var, alpha, omega, beta, sigma)

rao_black_traj = rao_blackwellisation(particles, weights, zetas, n_conditional_nodes)

errors_function_of_mcmc_step = []
errors_function_of_state_step = []
for r in range(1, (n_steps+1)):
    errors_function_of_mcmc_step.append(compute_error(rao_black_traj, true_means, r))

for t in range(1, (t_max+1)):
    errors_function_of_state_step.append(compute_error(rao_black_traj, true_means, state_step=t))

In [6]:
errors_function_of_state_step

[0.3688515928237122,
 0.2761470221866041,
 0.21321855403115264,
 0.17755261369705375,
 0.15314406304965547,
 0.16648230949920395,
 0.205608010381481,
 0.19228652075904637,
 0.21119470063050608,
 0.3154178824612124,
 0.4155884215376102,
 0.5149025844876897,
 0.6618559931133046,
 0.7890790115156365,
 0.9358789937910965,
 0.9457981276261777,
 0.984141844712793,
 1.074747845951816,
 1.2190781328401392,
 1.2508610237480597,
 1.2260258751211353,
 1.2745099324455424,
 1.385018915016812,
 1.3871075653920328,
 1.4276801983246685,
 1.451826024096275,
 1.4181119495277565,
 1.3725338052073133,
 1.3345709316081888,
 1.3397113172359763,
 1.3365120746450334,
 1.3246750908892764,
 1.3062443835907427,
 1.2902223716917873,
 1.271303310774959,
 1.2444057104296562,
 1.210968773267793,
 1.1961398208104794,
 1.1752438837024544,
 1.152876525739712,
 1.1382160647483335,
 1.1381254831729082,
 1.1148603880461474,
 1.0903109413607983,
 1.0675169949319205,
 1.0563250336625247,
 1.0419343336324292,
 1.027062951995